COCONUT AGE CLASSFICATION

## IMPORTS

In [33]:
import sys

!{sys.executable} -m pip install opencv-python
!"{sys.executable}" -m pip install opencv-python
%pip install tensorflow

'c:\Users\Mary' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tensorflow in c:\users\mary vee\appdata\local\programs\python\python313\lib\site-packages (2.20.0)




[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import cv2
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tensorflow.keras import layers

## PREPROCESSING

In [ ]:
# direction in which the folder where images are stored
image_dir = 'images' #this  depend kung asa siya gi butang
IMG_SIZE = (224, 224)


# ----------------------------------
# IMAGE PREPROCESSING
#-----------------------------------

img_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),   # Flipping
    layers.RandomRotation(0.2),                     # Rotation
    layers.RandomZoom(0.2),                         # Zooming
    layers.RandomContrast(0.2),                     # Contrast
    layers.RandomTranslation(0.1, 0.1),             # Shifting
    layers.Resizing(224, 224),                      # Resize to MobileNet input size
    layers.Rescaling(1./255, offset=-1),            # Normalization to [-1,1]
    layers.CenterCrop(224, 224)
])
   


In [40]:
# Load metadata with correct header row
df = pd.read_csv('Numerical_Datasets.csv', header=1) # Use the second row as header


# ----------------------------------
# NUMERICAL PREPROCESSING
#-----------------------------------

# Step 1: Applying the PCA Formula: Age = TrunkHeight / L11 + 5
df["age_formula"] = df["HeightTrunk"] / df["11leafscars"] + 5

# Step 2: Apply the General regression
aprox_age = "Year" in df.columns and df["Year"].notna().any()

if aprox_age:
    df["actual_age"] = 2025 - df["Year"]
    mask = df["actual_age"].notna() & np.isfinite(df["actual_age"])
    x = df.loc[mask, ["age_formula"]]
    y = df.loc[mask, ["actual_age"]]
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(x, y)
    a = float(reg.coef_[0])
    b = float(reg.intercept_)
else:
    # No labels at all, it will fall back to identity (no correction needed)
    a, b = 1.0, 0.0

# Step 3: Apply the linear regression formula a * (age_formula) + b (without
# planting year)

# Step 4: Show the results
print("Calibrated a, b:", a, b)
print(df[["age_formula", "actual_age"]])

Calibrated a, b: 2.7072163520770096 -10.558086853825515
     age_formula  actual_age
0      20.233333          18
1      20.233333          18
2      20.233333          18
3      19.061538          18
4      19.061538          18
..           ...         ...
405    15.252101          49
406    13.776978          49
407    13.776978          49
408    13.776978          49
409    13.776978          49

[410 rows x 2 columns]


C:\Users\Mary Vee\AppData\Local\Temp\ipykernel_2584\1104961044.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  a = float(reg.coef_[0])
C:\Users\Mary Vee\AppData\Local\Temp\ipykernel_2584\1104961044.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b = float(reg.intercept_)
